# Camera Based Occupancy Sensing

### Define CNN Models

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

def load_resnet50(num_classes):
    model =  models.resnet50(weights='ResNet50_Weights.DEFAULT')
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, num_classes)
    return model

def load_efficientnet_b0(num_classes):
    model = models.efficientnet_b0(weights='EfficientNet_B0_Weights.DEFAULT')
    num_features = model.classifier[1].in_features
    model.classifier[1] = nn.Linear(num_features, num_classes)
    return model

def load_mobilenet_v2(num_classes):
    model = models.mobilenet_v2(weights='MobileNet_V2_Weights.DEFAULT')
    num_features = model.classifier[1].in_features
    model.classifier[1] = nn.Linear(num_features, num_classes)
    return model

def load_mobilenet_v3_large(num_classes):
    model = models.mobilenet_v3_large(weights='MobileNet_V3_Large_Weights.DEFAULT')
    num_features = model.classifier[3].in_features
    model.classifier[3] = nn.Linear(num_features, num_classes)
    return model

def load_mobilenet_v3_small(num_classes):
    model = models.mobilenet_v3_small(weights='MobileNet_V3_Small_Weights.DEFAULT')
    num_features = model.classifier[3].in_features
    model.classifier[3] = nn.Linear(num_features, num_classes)
    return model

## Model Testing Section

In [18]:
import torch

# Move to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [19]:
!pip install scikit-plot

### Load Test Dataset

In [20]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

def create_test_dataloader(data_dir, batch_size, num_workers=4):
    # Define transformations for the test dataset
    data_transform = transforms.Compose([
        transforms.Resize((244, 244)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    # Create the test dataset
    test_dataset = datasets.ImageFolder(os.path.join(data_dir, 'test'), transform=data_transform)

    # Create the test dataloader
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

    return test_loader

data_dir = 'Dataset/'
batch_size = 32
test_loader = create_test_dataloader(data_dir, batch_size)

In [21]:
import torch
import torch.nn as nn
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import scikitplot as skplt

# Function to load the model
def load_model(model_class, checkpoint_path, num_classes):
    model = model_class(num_classes=num_classes)
    model.load_state_dict(torch.load(checkpoint_path, map_location=device))
    model.eval()
    return model
    
def evaluate_model(model, test_loader, criterion, save_path):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc='Testing'):
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs).squeeze(1)  # Output shape: [batch_size]
            loss = criterion(outputs, labels.float())  # Convert labels to float

            test_loss += loss.item() * inputs.size(0)

            # Apply sigmoid activation and classify based on threshold 0.5
            predicted = (torch.sigmoid(outputs) >= 0.5).float()

            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    test_loss = test_loss / len(test_loader.dataset)
    test_acc = correct / total
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc*100:.2f}%')

In [22]:
# List of model loading functions
model_functions = [
    load_resnet50,
    load_mobilenet_v3_large,
    load_efficientnet_b0,
    load_mobilenet_v2,
    load_mobilenet_v3_small,
]

checkpoint_paths = [
    'Checkpoints/resnet50/model_epoch_6.pth',
    'Checkpoints/mobilenet_v3_large/model_epoch_3.pth',
    'Checkpoints/efficientnet_b0/model_epoch_5.pth',
    'Checkpoints/mobilenet_v2/model_epoch_5.pth',
    'Checkpoints/mobilenet_v3_small/model_epoch_6.pth',
]

In [23]:
num_classes = 1
loss_fn = nn.BCEWithLogitsLoss()

for get_model, checkpoint_path in zip(model_functions, checkpoint_paths):
    # Load the model using the function
    loaded_model = load_model(get_model, checkpoint_path, num_classes)
    loaded_model = loaded_model.to(device)
    # Evaluate the loaded model on the test set
    print(checkpoint_path)
    evaluate_model(loaded_model, test_loader, loss_fn, checkpoint_path.split('/')[1])

Checkpoints/resnet50/model_epoch_6.pth


Testing: 100%|███████████████████████████████████████████████████████████████████████████| 3/3 [00:04<00:00,  1.36s/it]


Test Loss: 0.2154, Test Accuracy: 89.25%
Checkpoints/mobilenet_v3_large/model_epoch_3.pth


Testing: 100%|███████████████████████████████████████████████████████████████████████████| 3/3 [00:03<00:00,  1.31s/it]


Test Loss: 0.3350, Test Accuracy: 94.62%
Checkpoints/efficientnet_b0/model_epoch_5.pth


...

Test Loss: 0.1918, Test Accuracy: 90.32%
Checkpoints/mobilenet_v2/model_epoch_5.pth


Testing: 100%|███████████████████████████████████████████████████████████████████████████| 3/3 [00:03<00:00,  1.28s/it]


Test Loss: 0.1205, Test Accuracy: 97.85%
Checkpoints/mobilenet_v3_small/model_epoch_6.pth


Testing: 100%|███████████████████████████████████████████████████████████████████████████| 3/3 [00:03<00:00,  1.25s/it]

Test Loss: 0.8295, Test Accuracy: 88.17%
